In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [7]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

#reduce number of sentences for faster processing
alice_sents = alice_sents[0:250]
persuasion_sents = persuasion_sents[0:250]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [8]:
sentences.shape

(500, 2)

In [9]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 10 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [10]:
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 10
Processing row 20
Processing row 30
Processing row 40
Processing row 50
Processing row 60
Processing row 70
Processing row 80
Processing row 90
Processing row 100
Processing row 110
Processing row 120
Processing row 130
Processing row 140
Processing row 150
Processing row 160
Processing row 170
Processing row 180
Processing row 190
Processing row 200
Processing row 210
Processing row 220
Processing row 230
Processing row 240
Processing row 250
Processing row 260
Processing row 270
Processing row 280
Processing row 290
Processing row 300
Processing row 310
Processing row 320
Processing row 330
Processing row 340
Processing row 350
Processing row 360
Processing row 370
Processing row 380
Processing row 390
Processing row 400
Processing row 410
Processing row 420
Processing row 430
Processing row 440
Processing row 450
Processing row 460
Processing row 470
Processing row 480
Processing row 490


,hospitality,servant,speed,grumble,unquestionably,creature,comprehend,disappear,jug,actually,...,good,adjourn,entitle,angry,coast,inclination,bleed,scratch,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [11]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9933333333333333

Test set score: 0.75


In [12]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(300, 3062) (300,)
Training set score: 0.9933333333333333

Test set score: 0.83


In [13]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9866666666666667

Test set score: 0.775


In [14]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [15]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [16]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:250]

In [17]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 10
Processing row 20
Processing row 30
Processing row 40
Processing row 50
Processing row 60
Processing row 70
Processing row 80
Processing row 90
Processing row 100
Processing row 110
Processing row 120
Processing row 130
Processing row 140
Processing row 150
Processing row 160
Processing row 170
Processing row 180
Processing row 190
Processing row 200
Processing row 210
Processing row 220
Processing row 230
Processing row 240
done


In [18]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.44554455445544555


col_0,Austen,Carroll
row_0,,
Austen,108,142
Carroll,82,72


### Note on sample
Using only the first 250 sentences of each document reducing the accuracy of the model to predict the author of another text (In this case, Emma).  The test set score for the full text was 70%, while the sample is just 45%

## Challenge 0:
Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to improve performance. Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires. Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.

In [20]:
# Random Forest model

print('\nTest set score:', rfc.score(X_Emma_test, y_Emma_test))
rfc_Emma_predicted = rfc.predict(X_Emma_test)
pd.crosstab(y_Emma_test, rfc_Emma_predicted)


Test set score: 0.4207920792079208


col_0,Austen,Carroll
row_0,,
Austen,97,153
Carroll,81,73


### SVM Model

In [22]:
# SVM Model
from sklearn import svm

svc = svm.SVC()
train = svc.fit(X_train, y_train)

print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.5133333333333333

Test set score: 0.48


In [23]:
print('\nTest set score:', svc.score(X_Emma_test, y_Emma_test))
svc_Emma_predicted = svc.predict(X_Emma_test)
pd.crosstab(y_Emma_test, svc_Emma_predicted)


Test set score: 0.3811881188118812


col_0,Carroll
row_0,
Austen,250
Carroll,154


### Add feature for sentence length

In [30]:
# add feature of sentence length

word_counts['sentence_length'] = len(word_counts['text_sentence'])
emma_bow['sentence_length'] = len(emma_bow['text_sentence'])

In [31]:
# redefine target and features, then test and train sets

Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

In [32]:
# train and test logistric regression model 

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(300, 3063) (300,)
Training set score: 0.9933333333333333

Test set score: 0.825


In [33]:
# apply logisitic regression model
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.5074257425742574


col_0,Austen,Carroll
row_0,,
Austen,133,117
Carroll,82,72


## Count parts of speech in sentence

In [100]:
# get list of pos in docs to create variables
part_list = []

for sentence in (word_counts['text_sentence']):
        
        # iterate over token in sentence
        for token in sentence:
            
            if not token.pos_ in part_list:    
                part_list.append(token.pos_)
                
print(part_list)

['PROPN', 'VERB', 'PART', 'ADV', 'ADJ', 'ADP', 'NOUN', 'DET', 'PUNCT', 'CCONJ', 'PRON', 'INTJ', 'NUM']


In [101]:
# count parts of speech

def count_pos(sentences):
    
    # create dataframe to collect pos info
    df = pd.DataFrame(columns=part_list)

    # iterate over each sentence
    for i, sentence in enumerate(sentences):
        
        # iterate over token in sentence
        for token in sentence:
            df[token.pos_] += 1
            
        
        if i % 10 == 0:
            print("Processing row {}".format(i))
            
    return df

In [102]:
pos_counts = count_pos(word_counts['text_sentence'])

Processing row 0
Processing row 10
Processing row 20
Processing row 30
Processing row 40
Processing row 50
Processing row 60
Processing row 70
Processing row 80
Processing row 90
Processing row 100
Processing row 110
Processing row 120
Processing row 130
Processing row 140
Processing row 150
Processing row 160
Processing row 170
Processing row 180
Processing row 190
Processing row 200
Processing row 210
Processing row 220
Processing row 230
Processing row 240
Processing row 250
Processing row 260
Processing row 270
Processing row 280
Processing row 290
Processing row 300
Processing row 310
Processing row 320
Processing row 330
Processing row 340
Processing row 350
Processing row 360
Processing row 370
Processing row 380
Processing row 390
Processing row 400
Processing row 410
Processing row 420
Processing row 430
Processing row 440
Processing row 450
Processing row 460
Processing row 470
Processing row 480
Processing row 490


In [104]:
p_w_counts = pd.concat([word_counts, pos_counts])

p_w_counts.head()

,'s,-PRON-,1,ADJ,ADP,ADV,CCONJ,DET,INTJ,NOUN,...,you,you'd,young,your,yours,youth,zeal,zealand,zealous,zigzag
0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [107]:
emma_pos_counts = count_pos(emma_bow['text_sentence'])

p_w_emma = pd.concat([emma_bow, emma_pos_counts])

p_w_emma.head()

Processing row 0
Processing row 10
Processing row 20
Processing row 30
Processing row 40
Processing row 50
Processing row 60
Processing row 70
Processing row 80
Processing row 90
Processing row 100
Processing row 110
Processing row 120
Processing row 130
Processing row 140
Processing row 150
Processing row 160
Processing row 170
Processing row 180
Processing row 190
Processing row 200
Processing row 210
Processing row 220
Processing row 230
Processing row 240


,'s,-PRON-,1,ADJ,ADP,ADV,CCONJ,DET,INTJ,NOUN,...,you,you'd,young,your,yours,youth,zeal,zealand,zealous,zigzag
0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,0,0,0,0,0
2,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [105]:
# redefine target and features, then test and train sets

Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

ValueError: all the input array dimensions except for the concatenation axis must match exactly

## Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

In [78]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [86]:
parents = gutenberg.raw('edgeworth-parents.txt')

In [88]:
print('\nRaw:\n', parents[0:200])


Raw:
 [The Parent's Assistant, by Maria Edgeworth]


THE ORPHANS.

Near the ruins of the castle of Rossmore, in Ireland, is a small cabin,
in which there once lived a widow and her four children.  As 


In [89]:
# The Chapter indicator is idiosyncratic
parents = re.sub(r'[A-Z]+', '', parents)
    
parents = text_cleaner(parents)

In [90]:
parents_doc = nlp(parents)

In [94]:
parents_sents = [[sent, "Edgeworth"] for sent in parents_doc.sents]
parents_sents = parents_sents[0:250]

In [95]:
p_sentences = pd.DataFrame(parents_sents)
p_sentences.head()

,0,1
0,"(., ear, the, ruins, of, the, castle, of, ossm...",Edgeworth
1,"(s, long, as, she, was, able, to, work)",Edgeworth
2,"(,, she, was, very, industrious, ,, and, was, ...",Edgeworth
3,"(ary, was, at, this, time, about, twelve, year...",Edgeworth
4,"(ne, evening, she, was, sitting, at, the, foot...",Edgeworth


In [96]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
parentwords = bag_of_words(parents_doc)

In [97]:
p_word_counts = bow_features(p_sentences, common_words)
p_word_counts.head()

Processing row 0


,hospitality,servant,speed,grumble,unquestionably,creature,comprehend,disappear,jug,actually,...,good,adjourn,entitle,angry,coast,inclination,bleed,scratch,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(., ear, the, ruins, of, the, castle, of, ossm...",Edgeworth
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(s, long, as, she, was, able, to, work)",Edgeworth
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,"(,, she, was, very, industrious, ,, and, was, ...",Edgeworth
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(ary, was, at, this, time, about, twelve, year...",Edgeworth
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(ne, evening, she, was, sitting, at, the, foot...",Edgeworth
